# Webscraping the codebooks

In [3]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
import urllib3
import regex
import re
import os

To get the codebooks I need to:
- request the codebook html page
- get the html
- pull out the name for each variable
- drop them in a list
- pull out the description for each variable
- drop them in a list
- merge those lists into a dataframe

From there, I will need to:
- get the links for each variable name
- request each page
- get the values from the value column
- drop them in a list
- get the values from the unweighted column
- drop them in a list
- get the values from the weighted value column
- drop them in a list
- merge those lists into a dataframe

Finally:
- join the variable dfs onto the var+description df in a way where the variable name and description are repeated for each value pertaining to them

Then I will be able to use the codebooks to substitute the codes in the data for meaningful variable names with descriptions. Hopefully. Lol

### *Full Year Consolidated Data*

In [12]:
# define function to retrieve the links from the page(collapse above)

def getPUFLinks(URL):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text)
    PUFlinks = []

    for link in soup.findAll('tr', attrs = {'id' : 'Row1_neoTD3'}):
        href = link.find('a').get('href')
        PUF = href.split('=')[1]
        if PUF in PUFids:
            PUFlinks.append(link.find('a').get('href'))

    return PUFlinks

In [13]:
#get the links for full consolidated data pages
PUFids = ['HC-233', 'HC-224', 'HC-216', 'HC-209', 'HC-201', 'HC-192', 'HC-181', 'HC-171', 'HC-163', 'HC-155']
URL = 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=1%2CHousehold+Full+Year+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Consolidated+Data'

fcd = getPUFLinks(URL)
print(fcd)

['download_data_files_detail.jsp?cboPufNumber=HC-233', 'download_data_files_detail.jsp?cboPufNumber=HC-224', 'download_data_files_detail.jsp?cboPufNumber=HC-216', 'download_data_files_detail.jsp?cboPufNumber=HC-209', 'download_data_files_detail.jsp?cboPufNumber=HC-201', 'download_data_files_detail.jsp?cboPufNumber=HC-192', 'download_data_files_detail.jsp?cboPufNumber=HC-181', 'download_data_files_detail.jsp?cboPufNumber=HC-171', 'download_data_files_detail.jsp?cboPufNumber=HC-163', 'download_data_files_detail.jsp?cboPufNumber=HC-155']


In [14]:
#create list of links to get pages with cb html links

PUFlinks = (['https://meps.ahrq.gov/mepsweb/data_stats/' + word for word in fcd])
print(PUFlinks)

['https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-233', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-224', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-216', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-209', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-201', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-192', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-181', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-171', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-163', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-155']


In [75]:
#define function to get links to codebooks
def getCBLinks(URL):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text)
    CBlinks = []
    
    for link in soup.findAll('tr', attrs = {'id' : 'faqRoll_neoTD3'}):
        href = link.findAll('a')[-1].get('href')
        prefix = 'download_data_files_codebook'
        if href.startswith(prefix) == True:
            CBlinks.append(link.findAll('a')[-1].get('href'))

    return CBlinks

In [76]:
#get the links to the codebook pages

cbls= []

for link in PUFlinks:
    cbls.extend(getCBLinks(link))
cbls

['download_data_files_codebook.jsp?PUFId=H233',
 'download_data_files_codebook.jsp?PUFId=H224',
 'download_data_files_codebook.jsp?PUFId=H216',
 'download_data_files_codebook.jsp?PUFId=H209',
 'download_data_files_codebook.jsp?PUFId=H201',
 'download_data_files_codebook.jsp?PUFId=H192',
 'download_data_files_codebook.jsp?PUFId=H181',
 'download_data_files_codebook.jsp?PUFId=H171',
 'download_data_files_codebook.jsp?PUFId=H163',
 'download_data_files_codebook.jsp?PUFId=H155']

In [77]:
cblinks = (['https://meps.ahrq.gov/mepsweb/data_stats/' + word for word in cbls])
print(cblinks)

['https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H233', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H224', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H216', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H209', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H201', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H192', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H181', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H171', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H163', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H155']


In [78]:
#need to turn this into a loop that runs over cblinks: 
# p1 = pd.read_html('https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H224')
# p1[1]

#need the loop to spit out a dataframe for each page with iterative names instead of overwriting each time like it's doing now
#instead of dumping output into empty list, create separate entities like 'cb_1'
for link in cblinks:
    page = pd.read_html(link)
    table = page[1]
table

KeyboardInterrupt: 

In [ ]:
#go to variable page and get text from value column(exclude "total" if possible)
#need to loop over this to access each variable page
'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H224&varName=' + p1['name']

#then do read_html to get those tables